In [19]:
# ! pip install gym

You are using pip version 18.0, however version 19.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import gym
import random
import numpy as np

In [3]:
env_name = 'CartPole-v1'
env = gym.make(env_name)

In [22]:
print(env.observation_space)
print(env.action_space)
print(env.action_space.n)

Box(4,)
Discrete(2)
2


In [25]:
state = env.reset()

for _ in range(200):
    
#     action = env.action_space.sample()
#     action_size = env.action_space.n
#     action = random.choice(range(action_size))
    
    pole_angle = state[2]
    actioin = 0 if pole_angle < 0 else 1
    
    state, reward, done, info = env.step(action)
    env.render()
    
env.close()

WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.


In [26]:
env_name = 'MountainCarContinuous-v0'
env = gym.make(env_name)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [31]:
print(env.observation_space)
print(env.action_space)
print(env.action_space.low, env.action_space.high)
print(env.action_space.shape)

Box(2,)
Box(1,)
[-1.] [1.]
(1,)


In [34]:
state = env.reset()

for _ in range(200):
    action = env.action_space.sample()
    state, reward, done, info = env.step(action)
    env.render()
    
env.close()

# Reinforcement Learning (Q-Learning)

# Q(st,at) = rt+1 + G.Qmax(st+1)

In [4]:
import gym 
import random 
import numpy as np
import time

from IPython.display import clear_output

In [5]:
from gym.envs.registration import register

try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'map_name' : '4x4', 'is_slippery': False},
        max_episode_steps=100,
        reward_threshold=0.78, # optimum = .8196
    )

except:
    pass

env_name = 'FrozenLakeNoSlip-v0'
env = gym.make(env_name)

In [48]:
print(env.observation_space)
print(env.action_space)

type(env.action_space)

Discrete(16)
Discrete(4)


gym.spaces.discrete.Discrete

In [6]:
class Agent():
    def __init__(self, env):
        self.is_discrete = type(env.action_space)==gym.spaces.discrete.Discrete
        if self.is_discrete:
            self.action_size = env.action_space.n
        else:
            self.action_low = env.action_space.low
            self.action_high = env.action_space.high
            self.action_shape = env.action_space.shape
        
    def get_action(self, state):
        # action = env.action_space.sample
        if self.is_discrete:
                action = random.choice(range(self.action_size))
        else:
            action = np.random.uniform(self.action_low, 
                                      self.action_high,
                                      self.action_shape)
        return action

In [7]:
agent = Agent(env)

In [73]:
for ep in range(2):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)

        env.render()
        time.sleep(0.05)
        clear_output(wait=True)
        

  (Down)
SFFF
FHFH
FFFH
HFFG


In [86]:
class QAgent(Agent):
    
    def __init__(self, env, discount_rate=0.97, learning_rate=0.01):
        super().__init__(env)
        self.state_size = env.observation_space.n
        self.exploration_rate = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        self.q_table = 1e-4 * np.random.random([self.state_size, self.action_size])
    
    def get_action(self, state):
        q_state = self.q_table[state]
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)
        return action_random if random.random() < self.exploration_rate else action_greedy
    
    def train(self, experience):
        state, action, next_state, reward, done = experience
        
        q_next = self.q_table[next_state]
        q_next = np.zeros([self.action_size]) if done else q_next
        q_target = reward + self.discount_rate*np.max(q_next)
        
        q_update = q_target - self.q_table[state, action]
        self.q_table[state, action] += self.learning_rate*q_update
        
        if done:
            self.exploration_rate *= 0.99

In [87]:
agent = QAgent(env)

In [116]:
total_reward = 0.0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train((state, action, next_state, reward, done))
        
        print(f"s:{state} a:{action}")
        print(f"Episode:{ep} Reward:{total_reward} Explore:{agent.exploration_rate}")
        
        env.render()
        
        print(agent.q_table)
        
        time.sleep(0.05)
        clear_output(wait=True)

s:0 a:1
Episode:99 Reward:0.0 Explore:0.21272570322901874
  (Down)
SFFF
FHFH
FFFH
HFFG
[[1.47846857e-04 7.30576274e-05 1.18005309e-04 1.47859444e-04]
 [2.38664456e-05 4.89742657e-05 9.06914723e-05 3.69911294e-05]
 [2.24507464e-05 1.57687796e-06 8.08555927e-05 9.45883793e-05]
 [2.97522887e-05 7.02327064e-05 6.57184184e-05 9.25299051e-05]
 [7.92818706e-05 6.66391778e-05 7.80079359e-05 4.05681149e-05]
 [4.79938664e-06 6.08446470e-05 6.80903735e-05 9.41491863e-05]
 [3.72264827e-05 5.50982073e-05 8.54649380e-05 5.86563247e-05]
 [3.80707730e-05 5.60392518e-06 2.08854415e-05 5.83312977e-05]
 [9.25225278e-05 1.72512429e-05 4.08168314e-06 9.70519336e-05]
 [6.70335537e-05 3.14600974e-05 7.23799412e-06 2.18630565e-05]
 [4.69992056e-05 3.77199465e-05 2.30951312e-06 8.25925344e-05]
 [3.66549997e-05 1.78715391e-05 2.07296534e-05 1.86198185e-05]
 [7.09503234e-05 3.03849791e-05 9.24381271e-05 8.24962281e-05]
 [7.60753256e-05 5.67712969e-06 2.50542430e-05 9.62964682e-05]
 [3.56453983e-05 9.86784779e-05

# Q-Learning using Neural Nets

In [1]:
import tensorflow as tf

/home/amarpreetsingh/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [25]:
class QNAgent(Agent):
    def __init__(self, env, discount_rate=0.97, learning_rate=0.001):
        super().__init__(env)
        self.state_size = env.observation_space.n
        self.exploration_rate = 1.0
        self.discount_rate = discount_rate
        self.learning_rate = learning_rate
        
        self.build_model()
        
        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())
        
        
    def build_model(self):
        
        tf.reset_default_graph()
        
        self.state_in = tf.placeholder(tf.int32, shape=[1])
        self.action_in = tf.placeholder(tf.int32, shape=[1])
        self.target_in = tf.placeholder(tf.float32, shape=[1])
        
        self.state = tf.one_hot(self.state_in, depth=self.state_size)
        self.action = tf.one_hot(self.action_in, depth=self.action_size)
        
        self.q_state = tf.layers.dense(self.state, units=self.action_size, name='q_table')
        self.q_action = tf.reduce_sum(tf.multiply(self.q_state, self.action), axis=1)
        
        self.loss = tf.reduce_sum(tf.square(self.target_in - self.q_action))
        self.optimizer = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss)
        
    def get_action(self, state):
        q_state = self.sess.run(self.q_state, feed_dict={self.state_in: [state]})
        action_greedy = np.argmax(q_state)
        action_random = super().get_action(state)
        return action_random if random.random()<self.exploration_rate else action_greedy
    
    def train(self, experience):
        state, action, next_state ,reward, done = [[exp] for exp in experience ]
        
        q_next = self.sess.run(self.q_state, feed_dict={self.state_in: next_state})
        q_next[done] = np.zeros(self.action_size)
        q_target = reward + self.discount_rate*np.max(q_next)
        
        feed = {self.state_in:state, self.action_in:action, self.target_in:q_target}
        self.sess.run(self.optimizer, feed_dict=feed)
        
        if done[0]: 
            self.exploration_rate *= 0.99
            
    def __del__(self):
        self.sess.close()
        

In [26]:
agent = QNAgent(env)

In [30]:
total_reward = 0.0
for ep in range(100):
    state = env.reset()
    done = False
    while not done:
        action = agent.get_action(state)
        next_state, reward, done, info = env.step(action)
        agent.train((state, action, next_state, reward, done))
        state = next_state
        total_reward += reward
        
        print(f's:{state}, a:{action}')
        print(f"Episode:{ep} Reward:{total_reward} Explore:{agent.exploration_rate}")
        env.render()
        
        with tf.variable_scope("q_table", reuse=True):
            weights = agent.sess.run(tf.get_variable("kernel"))
            print(weights)
        time.sleep(0.05)
        clear_output(wait=True)

s:12, a:0
Episode:99 Reward:6.0 Explore:0.13397967485796175
  (Left)
SFFF
FHFH
FFFH
HFFG
[[ 1.4115499   2.8460217   0.823363   -0.21975632]
 [ 1.5008498  -2.6347911   1.3769156   0.669311  ]
 [ 0.8415626   0.7062567   1.2119367   0.2697629 ]
 [ 0.34555078 -1.4447845   1.3179134   0.629062  ]
 [ 0.117387    2.3890557  -0.9878063   1.5068276 ]
 [-0.3293431   0.0804112  -0.11855808  0.3902716 ]
 [-1.3848499   0.4552659  -0.11696961  0.9787792 ]
 [ 0.1918956  -0.25369915 -0.10539174  0.49528933]
 [ 0.4818096  -1.5581161   2.350788    0.4318273 ]
 [ 0.6181331   1.1841224   1.8554243  -2.162555  ]
 [ 0.9089021   0.7561973  -0.25447994  0.15577   ]
 [ 0.34934556 -0.05900297 -0.25172976  0.37717134]
 [-0.4705882  -0.5148027  -0.23669785 -0.2487303 ]
 [-0.48896223  0.65616924  1.8868823   1.5586889 ]
 [ 0.9198828   0.68129003  0.32311583  0.9682304 ]
 [-0.41099185  0.15661222 -0.23838988 -0.18105829]]
